In [1]:
from mass import MassClassifier
from scipy import sparse
import numpy as np
import pandas as pd

/home/home1/student/tianjian1/.conda/envs/ml/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
old=pd.read_csv("0.csv")

In [3]:
traindata=np.load("train_data.npy")
testdata=np.load("test_data.npy")
y=np.load("train_label.npy")
test_label=np.load("test_label.npy")
X1=traindata[(~(y==0).all(1))][:,2,:]
X2=traindata[((y==0).all(1))][:,2,:]
Y1=y[(~(y==0).all(1))]
mytrainfeature=pd.DataFrame(data= X1, columns=old.columns[1:])
mytrainlabel=pd.DataFrame(data= Y1, columns=["R1","R2","R3"])

In [4]:
mytotaldata=np.concatenate([X1,testdata[:,2,:]],axis=0)
mytotalfeature=pd.DataFrame(data= mytotaldata, columns=old.columns[1:])

In [5]:
for i in mytotalfeature.columns:
    if len(pd.unique(mytotalfeature[i]))>20:
        mytotalfeature[i]=pd.cut(mytotalfeature[i],bins=10)
        mytotalfeature[i]=mytotalfeature[i].cat.codes

In [6]:
data=pd.concat([mytotalfeature[:-600],mytrainlabel],axis=1)

In [7]:
testdf=mytotalfeature[-600:]

In [8]:
from pgmpy.models import BayesianNetwork
edges=[
        ("R1","feature15"),
        ("R1","feature13"),
        ("R2","feature13"),
        ("R2","feature19"),
        ("R2","feature61_0"),
        ("R2","feature61_1"),
        ("R2","feature61_2"), 
        ("R2","feature61_3"),
        ("R2","feature61_4"),
        ("R2","feature61_5"),
        ("R2","feature61_6"),
        ("R2","feature61_7"),
        ("R2","feature69_0"), 
        ("R2","feature69_1"),
        ("R2","feature69_2"),
        ("R2","feature69_3"),
        ("R2","feature69_4"), 
        ("R2","feature69_5"),
        ("R2","feature69_6"), 
        ("R2","feature28_0"), 
        ("R2","feature28_1"),
        ("R2","feature28_2"), 
        ("R2","feature28_3"),
        ("R2","feature28_4"),
        ("R2","feature28_5"),
        ("R2","feature28_6"),
        ("R2","feature28_7"),
        ("R2","feature36_0"),
        ("R2","feature36_1"),
        ("R2","feature36_2"), 
        ("R2","feature36_3"),
        ("R2","feature36_4"), 
        ("R2","feature36_5"),
        ("R2","feature36_6"),
        ("R2","feature36_7"),
        ("R3","feature60"),
        ("R3","feature61_0"),
        ("R3","feature61_1"),
        ("R3","feature61_2"),
        ("R3","feature61_3"),
        ("R3","feature61_4"),
        ("R3","feature61_5"),
        ("R3","feature61_6"),
        ("R3","feature61_7"),
        ("R3","feature69_0"),
        ("R3","feature69_1"),
        ("R3","feature69_2"),
        ("R3","feature69_3"),
        ("R3","feature69_4"),
        ("R3","feature69_5"),
        ("R3","feature69_6"),
        ("R3","feature20_0"),
        ("R3","feature20_1"),
        ("R3","feature20_2"),
        ("R3","feature20_3"),
        ("R3","feature20_4"),
        ("R3","feature20_5"),
        ("R3","feature20_6"),
        ("R3","feature20_7"),
        ("R3","feature28_0"),
        ("R3","feature28_1"),
        ("R3","feature28_2"),
        ("R3","feature28_3"),
        ("R3","feature28_4"),
        ("R3","feature28_5"),
        ("R3","feature28_6"),
        ("R3","feature28_7"),
        ("R3","feature36_0"),
        ("R3","feature36_1"),
        ("R3","feature36_2"),
        ("R3","feature36_3"),
        ("R3","feature36_4"),
        ("R3","feature36_5"),
        ("R3","feature36_6"),
        ("R3","feature36_7"),        
    ]
edgeinv= [(i[1],i[0]) for i in edges]
telecom_model = BayesianNetwork(
  edges#+gen_edge(featureY[:1],featureX[:1])
)


In [9]:
telecom_model.fit(data)

/home/home1/student/tianjian1/.conda/envs/ml/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [10]:
from pgmpy.inference import VariableElimination
infer = VariableElimination(telecom_model)

In [11]:
evidence_feature=[]
for i in edges:
    evidence_feature.append(i[1])

In [12]:
evidence_feature_R1=[]
for i in edges:
    if i[0]=="R1":
        evidence_feature_R1.append(i[1])
        
evidence_feature_R2=[]
for i in edges:
    if i[0]=="R2":
        evidence_feature_R2.append(i[1])

evidence_feature_R3=[]
for i in edges:
    if i[0]=="R3":
        evidence_feature_R3.append(i[1])

In [13]:
testdf=testdf.reset_index(drop=True)

In [14]:
i=0
aa=infer.map_query(['R1'],evidence= dict(zip(evidence_feature_R1,testdf[evidence_feature_R1][i:i+1].values.flatten())))
aa

{'R1': 1.0}

In [15]:
pred=np.zeros([600,3])
for i in range(600):
    aa=infer.map_query(['R1',"R2","R3"],evidence= dict(zip(evidence_feature,testdf[evidence_feature][i:i+1].values.flatten())))
    pred[i,0],pred[i,1],pred[i,2]=aa["R1"],aa["R2"],aa["R3"]

In [16]:
def score(pred,label):
    plus=np.sum(pred*label,axis=1)
    minus=np.sum(pred*(1-label),axis=1)
    return np.mean((plus-minus)/np.sum(label,axis=1))
score(pred,test_label)

0.7033333333333334